### Query Enhancement – Query Expansion Techniques

In a RAG pipeline, the quality of the query sent to the retriever determines how good the retrieved context is — and therefore, how accurate the LLM’s final answer will be.

That’s where Query Expansion / Enhancement comes in.

#### 🎯 What is Query Enhancement?
Query enhancement refers to techniques used to improve or reformulate the user query to retrieve better, more relevant documents from the knowledge base.
It is especially useful when:

- The original query is short, ambiguous, or under-specified
- You want to broaden the scope to catch synonyms, related phrases, or spelling variants

In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap

In [2]:
## step1 : Load and split the dataset
loader = TextLoader("langchain_crewai_dataset.txt")
raw_docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(raw_docs)


In [3]:
chunks

[Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='LangChain is an open-source framework designed for developing applications powered by large language models (LLMs). It simplifies the process of building, managing, and scaling complex chains of thought by abstracting prompt management, retrieval, memory, and agent orchestration. Developers can use'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='and agent orchestration. Developers can use LangChain to create end-to-end pipelines that connect LLMs with tools, APIs, vector databases, and other knowledge sources. (v1)'),
 Document(metadata={'source': 'langchain_crewai_dataset.txt'}, page_content='At the heart of LangChain lies the concept of chains, which are sequences of calls to LLMs and other tools. Chains can be simple, such as a single prompt fed to an LLM, or complex, involving multiple conditionally executed steps. LangChain makes it easy to compose and reuse chains using st

In [4]:
### step 2: Vector Store
embedding_model=HuggingFaceEmbeddings(model_name="D:\\model\\BAAI-bge-m3")
vectorstore=FAISS.from_documents(chunks,embedding_model)

## step 3:MMR Retriever
retriever=vectorstore.as_retriever(search_type="mmr",search_kwargs={"k":5})
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000276E52CD160>, search_type='mmr', search_kwargs={'k': 5})

In [16]:
## step 4 : LLM and Prompt

import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model("groq:meta-llama/llama-4-maverick-17b-128e-instruct")
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000276DE562490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000276DE561590>, model_name='meta-llama/llama-4-maverick-17b-128e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [22]:
# Query expansion
query_expansion_prompt = PromptTemplate.from_template("""
You are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.
                                                   
Original query: "{query}"

Expanded query: 
""")

query_expansion_chain=query_expansion_prompt| llm | StrOutputParser()
query_expansion_chain

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='\nYou are a helpful assistant. Expand the following query to improve document retrieval by adding relevant synonyms, technical terms, and useful context.\n\nOriginal query: "{query}"\n\nExpanded query: \n')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000276DE562490>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000276DE561590>, model_name='meta-llama/llama-4-maverick-17b-128e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser()

In [23]:
query_expansion_chain.invoke({"query":"Langchain memory"})

'To expand the query "Langchain memory," we\'ll add relevant synonyms, technical terms, and context to improve document retrieval. LangChain is a framework for developing applications powered by large language models (LLMs), and "memory" in this context likely refers to how these models or applications retain or recall information.\n\nExpanded query: \n"LangChain memory management OR LangChain context retention OR LangChain information recall OR LangChain persistent memory OR LangChain caching mechanisms OR LangChain stateful processing"\n\nLet\'s break down the components of the expanded query:\n\n1. **LangChain memory management**: This term is relevant for documents discussing how LangChain handles or manages memory, which could include strategies for optimizing the use of memory or dealing with memory constraints.\n\n2. **LangChain context retention**: "Context retention" is a synonym for memory in the context of LangChain, referring to the ability of LangChain applications or mode

In [24]:
# RAG answering prompt
answer_prompt = PromptTemplate.from_template("""
Answer the question based on the context below.

Context:
{context}

Question: {input}
""")

document_chain=create_stuff_documents_chain(llm=llm,prompt=answer_prompt)

In [25]:
# Step 5: Full RAG pipeline with query expansion
rag_pipeline = (
    RunnableMap({
        "input": lambda x: x["input"],
        "context": lambda x: retriever.invoke(query_expansion_chain.invoke({"query": x["input"]}))
    })
    | document_chain
)

In [26]:
# Step 6: Run query
query = {"input": "What types of memory does LangChain support?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

To expand the query "{'input': 'What types of memory does LangChain support?'}", we need to consider relevant synonyms, technical terms, and useful context that can help in retrieving more accurate and comprehensive documents. Here's an expanded version:

Expanded query: "{'input': 'What kinds of memory/storage does LangChain framework support? Including RAM, cache, or persistent storage like disk/memory/storage devices?'}" 

Or in a more detailed and structured form for a search or query system:

Expanded query: 
"({'input': 'LangChain memory support'} OR {'input': 'Types of memory LangChain'} OR {'input': 'LangChain storage options'} OR {'input': 'LangChain framework memory types'}) AND ({'input': 'RAM'} OR {'input': 'cache memory'} OR {'input': 'persistent storage'} OR {'input': 'disk storage'} OR {'input': 'memory devices'})"

This expansion includes:
1. **Synonyms and related terms**: Using terms like "kinds", "types", "storage", and "framework" to broaden the search.
2. **Technic

In [30]:
# Step 6: Run query
query = {"input": "CrewAI agents?"}
print(query_expansion_chain.invoke({"query":query}))
response = rag_pipeline.invoke(query)
print("✅ Answer:\n", response)

To expand the query "{'input': 'CrewAI agents?'}" for improved document retrieval, we can incorporate relevant synonyms, technical terms, and useful context. Here is a suggested expansion:

Expanded query: "{'input': '(CrewAI OR 'Crew AI' OR 'Multi-Agent Systems' OR 'Autonomous Agents') AND (agents OR 'artificial intelligence' OR 'AI models' OR 'collaborative AI')?}"

Here's how the expansion was constructed:

1. **Incorporating Synonyms and Variations**:
   - "CrewAI" is searched as is, but also with a space ("Crew AI") to account for possible variations in how the term is presented in documents.
   - The term "Multi-Agent Systems" is added as it is a relevant technical term closely related to the concept of CrewAI agents, assuming CrewAI refers to a system or framework involving multiple AI agents working together.

2. **Adding Technical Terms**:
   - "Autonomous Agents" is included because it is a technical term that likely relates to the functionality or characteristics of CrewAI a